<img src="https://raw.githubusercontent.com/brazil-data-cube/code-gallery/master/img/logo-bdc.png" align="right" width="64"/>

# <span style="color: #336699">Land use and land cover classification in the Brazilian Cerrado biome using Brazil Data Cube</span>
<hr style="border:2px solid #0077b9;">

<br/>

<div style="text-align: center;font-size: 90%;">
    Rolf E. O. Simões <sup><a href="mailto:rolf.simoes@inpe.br"><i class="far fa-lg fa-envelope"></i></a> <a href="https://orcid.org/0000-0003-0953-4132"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>, Alber H. Sanchez <sup><a href="mailto:alber.ipia@inpe.br"><i class="far fa-lg fa-envelope"></i></a> <a href="https://orcid.org/0000-0001-7966-2880"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>, Felipe M. Carlos <sup><a href="mailto:felipe.carlos@inpe.br"><i class="far fa-lg fa-envelope"></i></a> <a href="https://orcid.org/0000-0002-3334-4315"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>, Leonardo S. Vieira <sup><a href="mailto:leonardo.vieira@inpe.br"><i class="far fa-lg fa-envelope"></i></a> <a href="https://orcid.org/0000-0002-3397-6232"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>,<br/>
    Karine R. Ferreira <sup><a href="mailto:karine.ferreira@inpe.br"><i class="far fa-lg fa-envelope"></i></a> <a href="https://orcid.org/0000-0003-2656-5504"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>, Lubia Vinhas <sup><a href="mailto:lubia.vinhas@inpe.br"><i class="far fa-lg fa-envelope"></i></a> <a href="https://orcid.org/0000-0003-1104-3607"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>, Gilberto R. Queiroz<sup>* <a href="mailto:gilberto.queiroz@inpe.br"><i class="far fa-lg fa-envelope"></i></a> <a href="https://orcid.org/0000-0001-7534-0219"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>
    <br/><br/>
    Earth Observation and Geoinformatics Division, National Institute for Space Research (INPE)
    <br/>
    Avenida dos Astronautas, 1758, Jardim da Granja, São José dos Campos, SP 12227-010, Brazil
    <br/><br/>
    <sup>*</sup> Author to whom correspondence should be addressed.
    <br/><br/>
    February 24, 2021
</div>

<br/>

<div style="text-align: justify;  margin-left: 10%; margin-right: 10%;">
<b>Abstract.</b> This Jupyter Notebook compendium contains useful information for the creation of land use and land cover (LULC) maps using Earth observations data cubes and machine learning (ML) techniques. The code is based on the research pipeline described in the paper <em>Earth Observation Data Cubes for Brazil: Requirements, Methodology and Products</em>. These notebooks access open data available in the Brazil Data Cube platform.
</div>    

<br/>
<div style="text-align: justify;  margin-left: 15%; margin-right: 15%;font-size: 75%; border-style: solid; border-color: #0077b9; border-width: 1px; padding: 5px;">
    <b>This Jupyter Notebook is supplement to the <a href="https://www.mdpi.com/2072-4292/12/24/4033/htm#sec5-remotesensing-12-04033" target="_blank">Section 5</a> of the following paper:</b>
    <div style="margin-left: 10px; margin-right: 10px">
    Ferreira, K.R.; Queiroz, G.R.; Vinhas, L.; Marujo, R.F.B.; Simoes, R.E.O.; Picoli, M.C.A.; Camara, G.; Cartaxo, R.; Gomes, V.C.F.; Santos, L.A.; Sanchez, A.H.; Arcanjo, J.S.; Fronza, J.G.; Noronha, C.A.; Costa, R.W.; Zaglia, M.C.; Zioti, F.; Korting, T.S.; Soares, A.R.; Chaves, M.E.D.; Fonseca, L.M.G. 2020. Earth Observation Data Cubes for Brazil: Requirements, Methodology and Products. Remote Sens. 12, no. 24: 4033. DOI: <a href="https://doi.org/10.3390/rs12244033" target="_blank">10.3390/rs12244033</a>.
    </div>
</div>

# <span style="color: #336699">Extracting time series from sample locations</span>
<hr style="border:1px solid #0077b9;">

This document describes the process of extracting time series from EO data cubes associated with LULC samples. This information is required during the creation and training of the model based on a MultiLayer Perceptron neural network described later in the notebooks related to the classification process.

In general, at this stage, as shown in the Figure 1, for each sample, a temporal spectral signature is associated considering different bands and vegetation indices. This allows each sample's labels to be linked to a specific temporal variation pattern, which is used by the algorithms for classification.

<div align="center">
  <img src="https://raw.githubusercontent.com/brazil-data-cube/code-gallery/master/img/bdc-article/land-trajectory.png" width="600px">
</div>
<br/>
<center><b>Figure 1</b> - Temporal patterns based on spectral signature.<br/><b>Source:</b> Victor Maus</center>

# <span style="color: #336699">The SITS R package</span>
<hr style="border:1px solid #0077b9;">


For the time series extraction process we use the [SITS R package](https://github.com/e-sensing/sits).It can be loaded with the `library` command:

In [ ]:
library(sits)

The access to the Brazil Data Cube (BDC) data products requires an **access key**. If you have an account for the Brazil Data Cube services, [sign in](https://brazildatacube.dpi.inpe.br/auth-app) and replace the key value in the code cell below. If you do not have an account, [sign up](https://brazildatacube.dpi.inpe.br/auth-app) to create an account and get the access key.

In [ ]:
MY_ACCESS_KEY <- "change-me"

Sys.setenv(BDC_ACCESS_KEY = MY_ACCESS_KEY)

## <span style="color: #336699">Crop Calendar</span>
<hr style="border:0.5px solid #0077b9;">

We use the region's agricultural calendar year that ranges from September 2018 to August 2019:

In [ ]:
start_date  <- "2018-09-01"
end_date    <- "2019-08-31"

## <span style="color: #336699">Sample Data Points</span>
<hr style="border:0.5px solid #0077b9;">

The sample set that will be used to train the MLP model described in the manuscript contains 922 data points organized into three classes:

* Crop (242)
* Natural Vegetation (422)
* Pasture (258)

The file `training-samples.csv` in the folder `training-samples` contains the labeled data points.

In [ ]:
sample_file <- "https://brazildatacube.dpi.inpe.br/geo-knowledge-hub/bdc-article/training-samples/training-samples.csv"

As one can see, this file contains the location (`latitude`/`longitude`), a timestamp (`start_date`/`end_date`) and a label:

In [ ]:
head( read.csv(sample_file) )

The SITS package contains some facilities to read the location of these samples and to extract the time series from data cubes. This process will result in a dataframe where each sample point is associated to time series from spectral bands and vegetation indices.

## <span style="color: #336699">Creating the Output Directory</span>
<hr style="border:0.5px solid #0077b9;">

Let's create an output folder for the datasets generated by this notebook. We are going to choose a folder named `training-samples`:

In [ ]:
output_dir <- "training-samples"

dir.create(
    path         = output_dir, 
    showWarnings = FALSE, 
    recursive    = TRUE
)

## <span style="color: #336699">Time Series Extraction</span>
<hr style="border:0.5px solid #0077b9;">

### <span style="color: #336699">CBERS-4/AWFI (16 days 'stack')</span>

The `sits_cube` function can be used to define the working data cube and the interest spectral bands and vegetation indices:

In [ ]:
cb4_cube <- sits_cube(
    type        = "BDC",
    url         = "https://brazildatacube.dpi.inpe.br/stac/",
    collection  = "CB4_64_16D_STK-1",
    start_date  = start_date,
    end_date    = end_date,
    bands       = c("BAND15", "BAND14", "BAND13", "BAND16", "NDVI", "EVI")
)

In the above cell we provided the following arguments

* `type`: Inform that the data cube will be based on the Brazil Data Cube definitions.

* `url`: The SpatioTemporal Asset Catalog address for BDC data cubes. In this sandbox you can use the following address: https://brazildatacube.dpi.inpe.br/stac/.

* `collection`: Defines the use of CBERS-4/AWFI data cube.

* `start_date` and `end_date`: The temporal interval of interest.

* `bands`: The list of spectral bands and spectral indices that will be accessed.

The `sits_get_data` retries the time series from the defined data cube `cb4_cube` using the locations in the sample file:

In [ ]:
cb4_samples_with_ts <- sits_get_data(cube = cb4_cube, file = sample_file)

The structure of the returned dataframe can be seen below:

In [ ]:
head(cb4_samples_with_ts, 2)

If you want to plot the NDVI time series for all the LULC classes, use the following `plot` function:

In [ ]:
plot(sits_select(cb4_samples_with_ts, bands = c("NDVI")))

If you are interested only in a single class:

In [ ]:
plot(sits_select(cb4_samples_with_ts[cb4_samples_with_ts$label == "Pasture",], bands = c("NDVI")))

Finally, let's save the dataframe with the samples and time series in a file named `CB4_64_16D_STK_1.rds`. We are going to use it later on the classification notebook.

In [ ]:
saveRDS(cb4_samples_with_ts, paste0(output_dir, "/CB4_64_16D_STK_1.rds"))

### <span style="color: #336699">Landsat-8/OLI (16 days 'stack')</span>

The following code cells repeat the process described on the previous section to the Landsat-8 data cube.

In [ ]:
l8_cube <- sits_cube(
    type        = "BDC",
    name        = "bdc-l8",
    url         = "https://brazildatacube.dpi.inpe.br/stac/",
    collection  = "LC8_30_16D_STK-1",
    start_date  = start_date,
    end_date    = end_date,
    bands       = c("band4", "band3", "band2", "band5", "NDVI", "EVI")
)

In [ ]:
l8_samples_with_ts <- sits_get_data(cube = l8_cube, file = sample_file)

saveRDS(l8_samples_with_ts, paste0(output_dir, "/LC8_30_16D_STK_1.rds"))

### <span style="color: #336699">Sentinel-2/MSI (16 days 'stack')</span>

The following code cells repeat the process described on the CBERS-4/AWFI data cube section to the Sentinel-2 data cube.

In [ ]:
s2_cube <- sits_cube(
    type        = "BDC",
    name        = "bdc-s2",
    url         = "https://brazildatacube.dpi.inpe.br/stac/",
    collection  = "S2_10_16D_STK-1",
    start_date  = start_date,
    end_date    = end_date,
    bands       = c("band4", "band3", "band2", "band8", "NDVI", "EVI")
)

In [ ]:
s2_samples_with_ts <- sits_get_data(cube = s2_cube, file = sample_file)

saveRDS(s2_samples_with_ts, paste0(output_dir, "/S2_10_16D_STK_1.rds"))

# <span style="color: #336699">Final Remarks</span>
<hr style="border:1px solid #0077b9;">

* A copy of the training samples with the time series for each satellite/sensor data cube can be found in the folder `training-samples/rds`.

* If you want to download and run this notebook in a workflow as a script, you can perform its parameterization through the [papermill library](https://github.com/nteract/papermill).

* The data cubes used to extract the time series are also available on [BDC-STAC](https://brazildatacube.dpi.inpe.br/stac/).